# Tool Calling

## Key Benefits

* **Enhanced Functionality**: Perform complex tasks like data retrieval, calculations, or command execution.
* **Structured Outputs**: Ensure outputs match predefined schemas for seamless data processing.
* **Dynamic Decision-Making**: Invoke appropriate tools based on conversation context for intelligent interactions.
* **Seamless Integration**: Enable smooth information flow across tools and components in workflows.
* **Improved User Experience**: Provide accurate, relevant responses using real-time data or calculations.
* **Flexibility & Customization**: Create custom tools to meet specific needs, enhancing application adaptability.

In [1]:
from langchain_openai import ChatOpenAI

/Users/thann/Library/CloudStorage/OneDrive-IndianInstituteofScience/Course_Contents/Deep Learning/Class Materials/Tutorials/deep-learning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
llm_model="gpt-4o-mini"
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [3]:
llm.invoke("How will the weather be in Bangalore today?")

AIMessage(content="I'm unable to provide real-time weather updates. For the latest weather information in Bangalore, I recommend checking a reliable weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bef2TsACAwUsISNS428jlBQVhMyNl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8f25fa82-9304-4636-9330-b3487a56c815-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

**LLM couldn't provide real-time weather updates**. Lets create a dummy tool to check whether in a specific city and allow LLMs to access these tools

In [4]:
from langchain_core.tools import tool

#defining different tools

# @tool decorator creates LangChain tools

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"  #this could be an external API, like Google Weather


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools) # allows llms to access tools

In [6]:
result = llm_with_tools.invoke("How will the weather be in Bangalore today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IelFjmICdXQBl08Jx7vUu7HR', 'function': {'arguments': '{"city":"Bangalore"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 157, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bef4aMzVGYI2qYKHc8LFMJHY2O6UE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--beb9329f-358d-42fb-a34a-ff803669f501-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Bangalore'}, 'id': 'call_IelFjmICdXQBl08Jx7vUu7HR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 157, 'output_tokens': 16, 'total_tokens': 173, 'input_to

In [7]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

messages = [
    HumanMessage(
        "How will the weather be in Bangalore today? Do you still have seats outdoor available?"
    )
]

result = llm_with_tools.invoke(messages)  #this only decides the tool to call
messages.append(result)

In [8]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Bangalore'},
  'id': 'call_Hsm0XbEy0REr4HjJzFvbcNHi',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Bangalore'},
  'id': 'call_KdFv5uMCqjPSfYppRP5wduUS',
  'type': 'tool_call'}]

In [9]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [10]:
from langchain_core.messages import ToolMessage

# Iterate over each tool call in the result's tool_calls list
for tool_call in result.tool_calls:

    # Retrieve the corresponding tool from the tool_mapping dictionary using the tool name (case insensitive)
    tool = tool_mapping[tool_call["name"].lower()]

    # Invoke the selected tool with the provided arguments and store the output
    tool_output = tool.invoke(tool_call["args"])

    # Create a ToolMessage object with the tool's output and the unique ID of the tool call
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [11]:
messages

[HumanMessage(content='How will the weather be in Bangalore today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hsm0XbEy0REr4HjJzFvbcNHi', 'function': {'arguments': '{"city": "Bangalore"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_KdFv5uMCqjPSfYppRP5wduUS', 'function': {'arguments': '{"city": "Bangalore"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 165, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bef5e6x7iL82tHbDpQY3KzzGHduAG', 'service_tier': 'default', 'finish_re

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Bangalore today is sunny with a temperature of 22°C. Additionally, outdoor seating is available at restaurants in the area.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 244, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bef8951NIQnwLrIMX4r8Al34kkpnX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c765d9e-0988-40c1-ab34-5ea8739bf01f-0', usage_metadata={'input_tokens': 244, 'output_tokens': 28, 'total_tokens': 272, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Math Tool

In [15]:
!pip install numexpr

In [13]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

In [16]:
llm = ChatOpenAI(model="gpt-4o-mini")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

## ReAct Template

ReAct is a general paradigm that **combines reasoning and acting with LLMs**. ReAct prompts LLMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while also enabling interaction to external environments (e.g., Wikipedia) to incorporate additional information into the reasoning. The figure below shows an example of ReAct and the different steps involved to perform question answering.

<img src="react.webp" width=500 height=500\>

In [17]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # Zero shot react prompt
                         verbose=True,
                         max_iterations=3)

/tmp/ipykernel_85579/1514226503.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools,


In [21]:
agent.invoke({"input":"How many members does the A Team have?"})



> Entering new AgentExecutor chain...
The A-Team is a fictional group from a TV series. The main members of the A-Team include four key characters: Colonel John "Hannibal" Smith, Lieutenant Templeton "Faceman" Peck, Captain H. M. "Howling Mad" Murdock, and Sergeant Bosco "B.A." Baracus. Therefore, I will conclude that the A-Team has 4 members. 

Final Answer: 4

> Finished chain.


{'input': 'How many members does the A Team have?', 'output': '4'}

In [23]:
agent.invoke({"input":"What is 100 multiplied by 100? What is the square root of that number?"})



> Entering new AgentExecutor chain...
I need to first calculate \( 100 \times 100 \) and then find the square root of the result. 
Action: Calculator 
Action Input: 100 * 100 
Observation: Answer: 10000
Thought:Now that I have the result of \( 100 \times 100 \), which is 10000, I need to calculate the square root of 10000. 
Action: Calculator 
Action Input: 10000**0.5 
Observation: Answer: 100.0
Thought:I now know the final answer
Final Answer: 100.0

> Finished chain.


{'input': 'What is 100 multiplied by 100? What is the square root of that number?',
 'output': '100.0'}